### Introduction to modules, layers, and models
> In this turtorial we will be learning how to build some simple tensorflow modules. <br> We will also learn to build model using keras api.

To build tensorflow model we will be using ***tf.Module*** class

In [35]:
import tensorflow as tf
import os 
from datetime import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4120)])
    except RuntimeError as e:
        print(e)

In [11]:
#### defining a Dense layer using tf.Module


class Dense(tf.Module):
    def __init__(self,in_features,out_features,name=None):
        super().__init__(name=name)
        self.w=tf.Variable(tf.random.normal([in_features,out_features]),name='w') ## shape==in_features,out_features
        self.b=tf.Variable(tf.zeros([out_features],name='b'))

    def __call__(self,x):
        y=tf.matmul(x,self.w)+self.b
        return tf.nn.relu(y)    


In [12]:
model=Dense(in_features=3,out_features=2,name='dense')

In [13]:
x=tf.constant([[1,2,3]],dtype=tf.float32)
model=Dense(3,2,'dense')
output=model(x)

In [30]:
print('model output : {}'.format(output))

model output : [[0.        6.8176084]]


In [14]:
print('input.shape: {}\noutput.shape: {}'.format(x.shape,output.shape))

input.shape: (1, 3)
output.shape: (1, 2)


In [15]:
## Seeing the trainables parameter of models
model.trainable_variables ## which is are bias and weight

(<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.5247791 ,  0.10138043],
        [-0.8302663 ,  0.953391  ],
        [ 0.60742116,  1.6031486 ]], dtype=float32)>)

In [51]:
### Creating Dense which can be build only from out_features

class FlexibleDense(tf.Module):
    def __init__(self,out_features,name=None):
        super().__init__(name=name)
        self.out_features=out_features
        self.is_built=False
    def __call__(self,x):
        self.in_features=x.shape[-1]
        if not self.is_built:
            self.w=tf.Variable(tf.random.normal([self.in_features,self.out_features]),name='w')
            self.b=tf.Variable(tf.zeros([self.out_features]),name='b')
            self.is_built=True
        y=tf.matmul(x,self.w)+self.b
        return tf.nn.relu(y)     

In [55]:
f_model=FlexibleDense(out_features=2)
output_f=f_model(x)

In [56]:
output_f

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.      , 3.180626]], dtype=float32)>

### Saving weights using ***checkpoints***
> You can save a tf.Module as both a ***checkpoint*** and a ***SavedModel*** <br>
>  Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules). <br>
> Checkpoints consist of two kinds of files: the data itself and an index file for metadata. The index file keeps track of what is actually saved and the numbering of checkpoints, while the checkpoint data contains the variable values and their attribute lookup paths.

In [17]:
## Saving a model as checkpoint 
chkp_path='checkpoint/my_checkpoint'
checkpoint=tf.train.Checkpoint(model=model)
checkpoint.write(chkp_path)

'checkpoint/my_checkpoint'

In [24]:
## Seeing what is inside of checkpoint
!ls checkpoint*

my_checkpoint.data-00000-of-00001  my_checkpoint.index


You can look inside a checkpoint to be sure the whole collection of variables is saved, sorted by the Python object that contains them.

In [26]:
tf.train.list_variables(chkp_path) ## Shows what variables are saved 

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [28]:
## loading the weight to new model using checkpoints 
new_model=Dense(in_features=3,out_features=2,name='new_dense')
new_checkpoint=tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("checkpoint/my_checkpoint")


In [31]:
## Feeding the same value x to new model
new_output=new_model(x)
print('new_model output : {}'.format(new_output))

new_model output : [[0.        6.8176084]]


In [33]:
## As the trainables weights value were restored the output from two models are same
new_output==output

<tf.Tensor: shape=(1, 2), dtype=bool, numpy=array([[ True,  True]])>

### Saving functions
TensorFlow can run models without the original Python objects, as demonstrated by TensorFlow Serving and TensorFlow Lite, even when you download a trained model from TensorFlow Hub.

TensorFlow needs to know how to do the computations described in Python, but without the original code. To do this, you can make a graph, which is described in the Introduction to graphs and functions guide.

This graph contains operations, or ops, that implement the function.

You can define a graph in the model above by adding the @tf.function decorator to indicate that this code should run as a graph.

In [39]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function ## Graph execution
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [40]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

2022-10-07 11:02:52.901419: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-10-07 11:02:52.901446: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-10-07 11:02:52.901641: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-10-07 11:02:52.901933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib:/usr/local/cuda-11.6/lib


tf.Tensor([[12.442682  2.187654]], shape=(1, 2), dtype=float32)


In [42]:
#docs_infra: no_execute
%tensorboard --logdir logs/func

Reusing TensorBoard on port 6006 (pid 111689), started 0:05:08 ago. (Use '!kill 111689' to kill it.)

### Saving weights as ***SavedModel***
> this is the recommended way of sharing completely trained models. Unlike checkpoints it contains both collection of fucntion as well as collection of weights.
> <br> tf.saved_model.save(my_model,'model_path')
> <br> The saved_model.pb file is a protocol buffer describing the functional tf.Graph.
> <br> Models and layers can be loaded from this representation without actually making an instance of the class that created it. This is desired in situations where you do not have (or want) a Python interpreter, such as serving at scale or on an edge device, or in situations where the original Python code is not available or practical to use.

In [34]:
## Saving the model
tf.saved_model.save(model,'saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


In [37]:
## what is inside of saved_model
os.listdir('saved_model/')

['variables', 'assets', 'saved_model.pb']

In [38]:
#what are inside of saved_model/variables
os.listdir('saved_model/variables/')

['variables.data-00000-of-00001', 'variables.index']